<a href="https://colab.research.google.com/github/isteiner93/home_assignment/blob/main/sentinel_logs_analysis_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# install_logs dataset analysis

## Libraries

In [ ]:
!pip install awswrangler pandas altair

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.0/379.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.2/85.2 kB 5.3 MB/s eta 0:00:00




---



## Athena Connection


In [ ]:
import os
import getpass

# Prompt for AWS credentials securely (no visible output)
os.environ["AWS_ACCESS_KEY_ID"] = getpass.getpass("Enter your AWS_ACCESS_KEY_ID: ")
os.environ["AWS_SECRET_ACCESS_KEY"] = getpass.getpass("Enter your AWS_SECRET_ACCESS_KEY: ")
os.environ["AWS_DEFAULT_REGION"] = "eu-north-1"


Enter your AWS_ACCESS_KEY_ID: ··········
Enter your AWS_SECRET_ACCESS_KEY: ··········




---



## Load table (dataset)


**I am already loading Athena cleaned & parsed data. So partial data validation has been done already in Glue Job**

Compared to the original .csv, data in Athena has gone through the following cleaning:

1.   Tries reading CSV using default comma , delimiter
2.   Checks for the presence of all required columns
3.   Cast `time` to timestamp.
4.   Cast `installation_succeed` to boolean.
5.   Cast `installation_duration_milliseconds` to bigint.
5.   DROP nulls `df.dropna(subset=["time", "id"])` --> only for critical fields (not all columns with NULLs)


In [ ]:
import awswrangler as wr
import pandas as pd

# Configuration - loading table
ATHENA_DATABASE = "s1_logs_db"
SQL_QUERY = """
SELECT *
FROM s1_logs_db.install_logs
"""

# Run Athena query
df = wr.athena.read_sql_query(
    sql=SQL_QUERY,
    database=ATHENA_DATABASE
)

# Preview the result
df.head(10)


,time,id,os,device,version,error_code,installation_duration_milliseconds,installation_succeed,partition_0,partition_1
0,2023-05-14 15:29:01.357,6f2cbe50-339e-49b2-967d-d8edfc26d1ae,Windows,Desktop,1.0.1,00,314863,True,glue_job,installations.parquet
1,2023-05-14 15:29:01.357,0d72e214-a8b0-4070-ae91-69243bdf2d0e,Mac OS,Laptop,2.0.3,00,167584,True,glue_job,installations.parquet
2,2023-05-14 15:29:01.357,6906889f-db03-4f7b-9e3c-fbe542598930,Linux,Desktop,1.0.2,00,785258,True,glue_job,installations.parquet
3,2023-05-14 15:29:01.357,679d112c-6b79-4443-be58-89942645c4c9,Linux,Server,1.0.2,00,866948,True,glue_job,installations.parquet
4,2023-05-14 15:29:01.357,9eb58561-1495-43e1-8eed-520e1c8e6714,Linux,Laptop,1.0.2,00,719791,True,glue_job,installations.parquet
5,2023-05-14 15:29:01.357,4c24eaaa-e986-4677-a9f1-bbfbbc5b7c23,Mac OS,Desktop,2.0.4,22,227370,False,glue_job,installations.parquet
6,2023-05-14 15:29:01.357,ac36ea9c-b819-40e1-8f29-63748e1d5e3f,Mac OS,Desktop,2.0.3,00,708917,True,glue_job,installations.parquet
7,2023-05-14 15:29:01.357,fecf2375-43da-427e-b39d-315599698c74,Linux,Desktop,1.0.2,00,161942,True,glue_job,installations.parquet
8,2023-05-14 15:29:01.357,0d72e214-a8b0-4070-ae91-69243bdf2d0e,Mac OS,Laptop,2.0.3,00,167584,True,glue_job,installations.parquet
9,2023-05-14 15:29:01.357,843ae8c1-a240-4d95-81da-e7d918160d12,Mac OS,Desktop,2.0.3,00,829500,True,glue_job,installations.parquet




---



## Data validation

### Data Profiling



Here are some basic findings:
1. `error_code` has 17 missing values - this is intended, but important to note.

2.  There are some ID duplicates (`985 unique IDs out of 994 rows`). Important to know the dataset is not unique on ID level & when transforming data further (e.g. in dbt) we should consider this & write proper validation tests for new models.

3. We can see negative time values for `installation_duration_milliseconds` (logically problematic). We should again consider this when writing further transformation during aggregating. We can also possibly drop these rows.

In [ ]:
# 1. Data Profiling and Summary
print("=== Data Profiling Summary ===")
print("\n--- Column Data Types ---")
print(df.dtypes)

print("\n--- Missing Values ---")
print(df.isnull().sum())

print("\n--- Unique Values ---")
print(df.nunique())

print("\n--- Basic Statistical Summary ---")
print(df.describe(include='all'))


=== Data Profiling Summary ===

--- Column Data Types ---
time                                  datetime64[ns]
id                                    string[python]
os                                    string[python]
device                                string[python]
version                               string[python]
error_code                            string[python]
installation_duration_milliseconds             Int64
installation_succeed                         boolean
partition_0                           string[python]
partition_1                           string[python]
dtype: object

--- Missing Values ---
time                                   0
id                                     0
os                                     0
device                                 0
version                                0
error_code                            17
installation_duration_milliseconds     0
installation_succeed                   0
partition_0                            0
parti



---



### Schema Validation

In the next step we can again **check if the data types of the table** (file) is as expected. We have done this already in the Glue Spark Job, but lets check again.

As seen below all the columns match expected schema columns ✅

In [ ]:
# 2. Schema Validation
print("\n=== Schema Validation ===")

# Defined schema as expected
expected_schema = {
    'time': 'datetime64[ns]',
    'id': 'string',
    'os': 'string',
    'device': 'string',
    'version': 'string',
    'error_code': 'string',
    'installation_duration_milliseconds': 'Int64',
    'installation_succeed': 'boolean'
}

# Schema validation
print("\n--- Schema Mismatches ---")
for col, expected_type in expected_schema.items():
    if col in df.columns:
        actual_type = str(df[col].dtype)
        if actual_type == expected_type:
            print(f"[Match] Column '{col}': Expected {expected_type}, Found {actual_type}")
        if actual_type != expected_type:
            print(f"[Mismatch] Column '{col}': Expected {expected_type}, Found {actual_type}")
    else:
        print(f"[Missing] Column '{col}' is not in the DataFrame")



=== Schema Validation ===

--- Schema Mismatches ---
[Match] Column 'time': Expected datetime64[ns], Found datetime64[ns]
[Match] Column 'id': Expected string, Found string
[Match] Column 'os': Expected string, Found string
[Match] Column 'device': Expected string, Found string
[Match] Column 'version': Expected string, Found string
[Match] Column 'error_code': Expected string, Found string
[Match] Column 'installation_duration_milliseconds': Expected Int64, Found Int64
[Match] Column 'installation_succeed': Expected boolean, Found boolean




---



### Data quality


Here we can confirm:
1. Again **17** empty `error_code` values
2. Again we see **6** records with `installation_duration_milliseconds < 0`.
3. No invalid booleans ✅

In [ ]:
# 3. Data Quality Checks
print("\n=== Data Quality Checks ===")

# Check 1: Null values in required fields
required_fields = list(expected_schema.keys())
null_issues = df[required_fields].isnull().sum()
print("\n--- Null Values in Required Fields ---")
print(null_issues[null_issues > 0])

# Check 2: Negative installation durations (invalid)
negative_durations = df[df['installation_duration_milliseconds'] < 0]
print("\n--- Negative Installation Durations ---")
print(negative_durations)

# Check 3: Non-boolean values in installation_succeed
invalid_bool = df[~df['installation_succeed'].isin([True, False, pd.NA])]
print("\n--- Invalid Boolean Entries in 'installation_succeed' ---")
print(invalid_bool)


=== Data Quality Checks ===

--- Null Values in Required Fields ---
error_code    17
dtype: int64

--- Negative Installation Durations ---
                       time                                    id      os  \
232 2023-05-14 15:29:01.361  759377ae-eeb4-4e86-9169-f89be04aac27   Linux   
262 2023-05-14 15:29:01.362  e98807b8-c444-4c6f-8037-fe212d5b1d54   Linux   
320 2023-05-14 15:29:01.363  ec91e6b1-2783-44ad-a009-aa624a461e06  Mac OS   
359 2023-05-14 15:29:01.364  1949aadb-9d73-4cf1-9c44-ab503da82f71  Mac OS   
570 2023-05-14 15:29:01.368  584ecc95-e633-41ac-a999-36b0de454b6f   Linux   
674 2023-05-14 15:29:01.370  3adbbc6e-5ad9-4059-a0e8-5dec764b108c  Mac OS   

      device version error_code  installation_duration_milliseconds  \
232  Desktop   1.0.2         00                                 -13   
262  Desktop   1.0.2         00                                 -13   
320  Desktop   2.0.3         00                                 -13   
359  Desktop   2.0.3         00     



---



## Saves as .parquet

In [ ]:
# Save as Parquet locally
df.to_parquet("install_logs.parquet", index=False)
print("File saved")

File saved


## Data Analysis

Lets start by `df.describe` the dataset.
This shows again few interesting facts about the dataset:
1. `time` spread is very small (few miliseconds)
2. negative values for `installation_duration_milliseconds` (-13)

In [ ]:
df.describe()

,time,installation_duration_milliseconds
count,994,994.0
mean,2023-05-14 15:29:01.366558464,531515.720322
min,2023-05-14 15:29:01.357000,-13.0
25%,2023-05-14 15:29:01.360999936,295171.0
50%,2023-05-14 15:29:01.366000128,524194.0
75%,2023-05-14 15:29:01.372000,762584.75
max,2023-05-14 15:29:01.376000,999774.0
std,NaN,266314.701661




---



### Logs Count Distribution
Below a **size distribution** of values across 3 dimentions (version, os, device).

As we can see version 2.0.3, macOS and Desktop being the biggest contributors to the dataset (largest count of logs)

In [ ]:
import pandas as pd
import altair as alt

# Prepare the count data
version_counts = df['version'].value_counts().reset_index()
version_counts.columns = ['version', 'count']

os_counts = df['os'].value_counts().reset_index()
os_counts.columns = ['os', 'count']

device_counts = df['device'].value_counts().reset_index()
device_counts.columns = ['device', 'count']

# Plot 1: Count of each version
chart_version = alt.Chart(version_counts).mark_bar().encode(
    x=alt.X('version:N', title='Version'),
    y=alt.Y('count:Q', title='Count'),
    tooltip=['version', 'count']
).properties(
    title='Count by Version'
)

# Plot 2: Count of each OS
chart_os = alt.Chart(os_counts).mark_bar().encode(
    x=alt.X('os:N', title='OS'),
    y=alt.Y('count:Q', title='Count'),
    tooltip=['os', 'count']
).properties(
    title='Count by OS'
)

# Plot 3: Count of each device
chart_device = alt.Chart(device_counts).mark_bar().encode(
    x=alt.X('device:N', title='Device'),
    y=alt.Y('count:Q', title='Count'),
    tooltip=['device', 'count']
).properties(
    title='Count by Device'
)

# Combine the charts horizontally
combined_chart = (chart_version | chart_os | chart_device).resolve_axis(
    x='independent'
).resolve_scale(
    y='shared'
)

combined_chart.display()


alt.HConcatChart(...)



---



### Installation success rate
Below of **installation sucess rate per os, per device**
Couple of findings:
1. version `2.0.4` has a significantly lower installation success rate
2. similarly macOS is the only os with error rate, but it also is the most common os

In [ ]:
import pandas as pd
import altair as alt

def plot_installation_success_rate(df, category_column):
    if category_column not in df.columns:
        print(f"Error: Column '{category_column}' not found in the DataFrame.")
        return

    # Group by the category and calculate the success rate
    summary_df = df.groupby(category_column)['installation_succeed'].agg(['sum', 'count']).reset_index()
    summary_df['success_rate'] = summary_df['sum'] / summary_df['count']

    # Create the bar chart
    chart = alt.Chart(summary_df).mark_bar().encode(
        x=alt.X(category_column, sort='-y'), # Sort bars by success rate descending
        y=alt.Y('success_rate', axis=alt.Axis(format='.1%')), # Format y-axis as percentage
        tooltip=[category_column, 'success_rate', 'sum', 'count']
    ).properties(
        title=f'Installation Success Rate by {category_column.capitalize()}'
    )

    return chart

# Example usage:
chart_os = plot_installation_success_rate(df, 'os')
chart_device = plot_installation_success_rate(df, 'device')
chart_version = plot_installation_success_rate(df, 'version')


# Combine the charts horizontally
combined_chart = (chart_version | chart_os | chart_device).resolve_axis(
    x='independent'
).resolve_scale(
    y='shared'
)

combined_chart.display()

alt.HConcatChart(...)



---



### Mean installation duration per categories
In this section, I am exploring the average time it takes to install the system per different categories.

As seen in the graphs, the longest on average it takes on Windows, Deskptop and for version 1.0.1

In [ ]:
def plot_installation_duration_charts(df, categories=['os', 'device', 'version']):
  # Ensure installation duration is positive for aggregation
  df_positive_duration = df[df['installation_duration_milliseconds'] >= 0].copy()

  charts = []
  for category in categories:
    if category not in df_positive_duration.columns:
      print(f"Warning: Category '{category}' not found in DataFrame. Skipping.")
      continue
    # Calculate mean of installation duration per category
    category_duration = df_positive_duration.groupby(category)['installation_duration_milliseconds'].mean().reset_index()
    category_duration.columns = [category, 'mean_duration_milliseconds']
    # Altair chart for mean duration per category
    chart = alt.Chart(category_duration).mark_bar().encode(
        x=alt.X(f'{category}:N', title=category.replace('_', ' ').title()),
        y=alt.Y('mean_duration_milliseconds:Q', title='Mean Installation Duration (milliseconds)'),
        tooltip=[category, 'mean_duration_milliseconds']
    ).properties(
        title=f'Mean Installation Duration per {category.replace("_", " ").title()}'
    )
    charts.append(chart)

  if charts:
    combined_chart = alt.hconcat(*charts).resolve_axis(
        x='independent'
    ).resolve_scale(
        y='shared'
    )
    combined_chart.display()
  else:
    print("No charts were generated.")

plot_installation_duration_charts(df)

alt.HConcatChart(...)



---



### Box Plot Installation Duration - check
To make sure we don't count in some large outliers when checking the mean average duration times, we can visualize it with **boxplot** per all categories.

As we can see, there aren't any significat outliers that would siginificantly impact the data analysis. The distribution is similar for all caregories.

In [ ]:
def plot_installation_duration_boxplots(df, categories=['os', 'device', 'version']):
  # Filter out negative installation durations as they are invalid
  df_positive_duration = df[df['installation_duration_milliseconds'] >= 0].copy()

  charts = []
  for category in categories:
    if category not in df_positive_duration.columns:
      print(f"Warning: Category '{category}' not found in DataFrame. Skipping.")
      continue

    # Create a box plot for installation duration per category
    chart = alt.Chart(df_positive_duration).mark_boxplot().encode(
        x=alt.X(f'{category}:N', title=category.replace('_', ' ').title()),
        y=alt.Y('installation_duration_milliseconds:Q', title='Installation Duration (milliseconds)'),
        tooltip=[category, 'installation_duration_milliseconds']
    ).properties(
        title=f'Box Plot of Installation Duration per {category.replace("_", " ").title()}'
    )
    charts.append(chart)

  if charts:
    combined_chart = alt.hconcat(*charts).resolve_axis(
        x='independent'
    ).resolve_scale(
        y='shared'
    )
    combined_chart.display()
  else:
    print("No charts were generated.")

plot_installation_duration_boxplots(df)

alt.HConcatChart(...)

### Distribution of installation duration
In this section we looked at the distribution of `installation_duration_milliseconds` across all categorical dimensions.

What is interesting to see here is that there is NOT a normal distribution (gausian distribution) and we see the times (frequency) being split similarly across all time blocks.

In [ ]:
# Filter out negative installation durations
df_filtered = df[df['installation_duration_milliseconds'] >= 0].copy()

# Convert milliseconds to seconds for better readability on the plot
df_filtered['installation_duration_seconds'] = df_filtered['installation_duration_milliseconds'] / 1000

# Create histograms for installation duration distributions

# Distribution by OS
chart_duration_os = alt.Chart(df_filtered).mark_bar().encode(
    alt.X('installation_duration_seconds:Q', bin=True, title='Installation Duration (seconds)'),
    alt.Y('count()', title='Frequency'),
    color='os:N',
    column=alt.Column('os:N', header=alt.Header(titleOrient="bottom", labelOrient="bottom"), title='Operating System')
).properties(
    title='Distribution of Installation Duration by OS'
)

# Distribution by Device
chart_duration_device = alt.Chart(df_filtered).mark_bar().encode(
    alt.X('installation_duration_seconds:Q', bin=True, title='Installation Duration (seconds)'),
    alt.Y('count()', title='Frequency'),
    color='device:N',
    column=alt.Column('device:N', header=alt.Header(titleOrient="bottom", labelOrient="bottom"), title='Device')
).properties(
    title='Distribution of Installation Duration by Device'
)

# Distribution by Version
chart_duration_version = alt.Chart(df_filtered).mark_bar().encode(
    alt.X('installation_duration_seconds:Q', bin=True, title='Installation Duration (seconds)'),
    alt.Y('count()', title='Frequency'),
    color='version:N',
    column=alt.Column('version:N', header=alt.Header(titleOrient="bottom", labelOrient="bottom"), title='Version')
).properties(
    title='Distribution of Installation Duration by Version'
)

# Combine charts vertically or display individually
chart_duration_os.display()
chart_duration_device.display()
chart_duration_version.display()


alt.Chart(...)

alt.Chart(...)

alt.Chart(...)